# Serve DeepSeek R1 (Distilled Llama 70B) using provisioned throughput

This notebook demonstrates how to download and register the DeepSeek R1 distilled Llama model in Unity Catalog and deploy it using a Foundation Model APIs provisioned throughput endpoint.

## Install the `transformers` library from HuggingFace

In [0]:
!pip install transformers==4.44.2 mlflow
%restart_python

## Download DeepSeek R1 distilled Llama 70B 

The following code downloads the DeepSeek R1 distilled Llama 70B model to your local machine.

In [0]:
dbutils.widgets.text("model_id", "deepseek-ai/DeepSeek-R1-Distill-Llama-70B", "Name of Huggingface Model")

model_id = dbutils.widgets.get("model_id")

In [0]:
import os

LOCAL_DISK_HF = "/local_disk0/hf_cache"
os.makedirs(LOCAL_DISK_HF, exist_ok=True)
os.environ["HF_HOME"] = LOCAL_DISK_HF
os.environ["HF_DATASETS_CACHE"] = LOCAL_DISK_HF
os.environ["TRANSFORMERS_CACHE"] = LOCAL_DISK_HF

In [0]:
from huggingface_hub import snapshot_download
snapshot_download(model_id)

## Register the downloaded model to Unity Catalog

The following code shows how to start and log a run that registers the downloaded model to Unity Catalog.

In [0]:
import mlflow
import transformers

my_uc_catalog = "main"
my_uc_schema = "msh"
uc_model_name = "deepseek_r1_distilled_llama70b_v1"

task = "llm/v1/chat"
model = transformers.AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

transformers_model = {"model": model, "tokenizer": tokenizer}

with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model=transformers_model,
        artifact_path="model",
        task=task,
        registered_model_name=f"{my_uc_catalog}.{my_uc_schema}.{uc_model_name}",
        metadata={
            "task": task,
            "pretrained_model_name": "meta-llama/Llama-3.3-70B-Instruct",
            "databricks_model_family": "LlamaForCausalLM",
            "databricks_model_size_parameters": "70b",
        },
    )

## Create a provisioned throughput endpoint for model serving

The following code shows how to create a provisioned throughput model serving endpoint to serve the Llama 70B that you downloaded and registered to Unity Catalog.

In [0]:
from mlflow.deployments import get_deploy_client


client = get_deploy_client("databricks")


endpoint = client.create_endpoint(
    name=uc_model_name,
    config={
        "served_entities": [{
            "entity_name": f"{my_uc_catalog}.{my_uc_schema}.{uc_model_name}",
            "entity_version": model_info.registered_model_version,
             "min_provisioned_throughput": 0,
             "max_provisioned_throughput": 9500,
            "scale_to_zero_enabled": True
        }],
        "traffic_config": {
            "routes": [{
                "served_model_name": f"{uc_model_name}-{model_info.registered_model_version}",
                "traffic_percentage": 100
            }]
        }
    }
)